In [ ]:
import os
import warnings
import re
from functools import partial
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import optuna
import polars as pl
import polars.selectors as cs

from catboost import CatBoostRegressor, MultiTargetCustomMetric
from xgboost import XGBRegressor

from numpy.typing import ArrayLike, NDArray
from scipy.optimize import minimize
from polars.testing import assert_frame_equal
from sklearn.base import BaseEstimator
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from colorama import Fore, Style, init
from tqdm import tqdm

# Color printing
def PrintColor(text: str, color = Fore.BLUE, style = Style.BRIGHT):
    "Prints color outputs using colorama using a text F-string"
    print(style + color + text + Style.RESET_ALL)

warnings.filterwarnings("ignore", message="Failed to optimize method")


#DATA_DIR = Path("/kaggle/input/child-mind-institute-problematic-internet-use")
DATA_DIR = Path("./data")

TARGET_COLS = [
    "PCIAT-PCIAT_01",
    "PCIAT-PCIAT_02",
    "PCIAT-PCIAT_03",
    "PCIAT-PCIAT_04",
    "PCIAT-PCIAT_05",
    "PCIAT-PCIAT_06",
    "PCIAT-PCIAT_07",
    "PCIAT-PCIAT_08",
    "PCIAT-PCIAT_09",
    "PCIAT-PCIAT_10",
    "PCIAT-PCIAT_11",
    "PCIAT-PCIAT_12",
    "PCIAT-PCIAT_13",
    "PCIAT-PCIAT_14",
    "PCIAT-PCIAT_15",
    "PCIAT-PCIAT_16",
    "PCIAT-PCIAT_17",
    "PCIAT-PCIAT_18",
    "PCIAT-PCIAT_19",
    "PCIAT-PCIAT_20",
    "PCIAT-PCIAT_Total",
    "sii",
]

FEATURE_COLS = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI', 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference', 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP', 'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec', 'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season', 'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-Season', 'PreInt_EduHx-computerinternet_hoursday', 'X_std', 'X_min', 'X_5', 'Y_std', 'Y_45', 'Z_min', 'anglez_std', 'anglez_min', 'anglez_max', 'light_max', 'battery_voltage_mean', 'battery_voltage_std']

FEATURE_COLS = [
    "Basic_Demos-Age",
    "PreInt_EduHx-computerinternet_hoursday",
    "SDS-SDS_Total_Raw",
    "Physical-Height",
    "FGC-FGC_TL",
    "Physical-Waist_Circumference",
    "BIA-BIA_FMI",
    "BIA-BIA_Fat",
    "Fitness_Endurance-Time_Sec",
    "Fitness_Endurance-Max_Stage",
    "FGC-FGC_CU_Zone",
]

In [ ]:
# Credit to Ravi Ramakrishnan for his great Preprocessing class
# https://www.kaggle.com/code/ravi20076/cmi2024-baseline-v2

class Preprocessor:
    "This class organizes the preprocessing steps for the train-test data into a single code block"
    
    def __init__(
        self, cat_imp_val : str= "missing", ip_path: str = DATA_DIR
    ):
        self.cat_imp_val = cat_imp_val
        self.ip_path     = ip_path
        
    def make_pqfile_cols(
        self, verbose: bool = False, label: str = "Train"
    )->pl.DataFrame:
        "This method collates the id level parquet files and creates the aggregation columns in a polars dataframe"

        cols = ["X", "Y", "Z", "enmo", "anglez", "light", "battery_voltage"]
        
        ip_path   = os.path.join(self.ip_path, f"series_{label.lower()}.parquet")
        all_files = os.listdir(ip_path)

        for file_nb, file in tqdm(enumerate(all_files)):
            df = \
            pl.scan_parquet(
                os.path.join(ip_path, file, f"part-0.parquet")
            ).select(pl.col(cols)).\
            collect().\
            describe(
                percentiles = np.arange(0.05, 0.95, 0.10)
            ).\
            filter(~pl.col("statistic").is_in(["count", "null_count"])).\
            unpivot(index = "statistic").\
            with_columns(
                pl.concat_str([pl.col("variable"), pl.col("statistic")],separator = "_",).alias("myvar")
            ).\
            with_columns(pl.col("myvar").str.replace(r"\%", "")).\
            select(["myvar", "value"]).\
            transpose(column_names = "myvar").\
            select(pl.all().shrink_dtype()).\
            with_columns(
                pl.Series("id", np.array(re.sub("id=", "", file)))
            )

            if file_nb == 0:
                op_df = df.clone()
            elif file_nb > 0:
                op_df = pl.concat([op_df, df], how = "vertical_relaxed")

                if verbose:
                    print(f"---> Shapes = {op_df.shape}")
                else:
                    pass
            del df

        PrintColor(f"---> {label} - shape = {op_df.shape}", color = Fore.CYAN)
        return op_df

    def pp_data(
        self, df: pl.DataFrame, label: str = "Train", cat_cols: list = [], 
    ):
        "This method preprocesses the train-test data with requisite steps"
        
        PrintColor(f"\n --- Data Processing - {label} --- \n")
        PrintColor(f"---> Shape = {df.shape} - memory usage {df.estimated_size('mb') :.3f} MB", 
                   color = Fore.CYAN
                  )
        
        if label == "Train":
            cat_cols = df.select(cs.string().exclude("id")).columns
        else:
            pass
        
        df    = df.with_columns(pl.col(cat_cols).fill_null(self.cat_imp_val).cast(pl.Categorical))
        op_df = self.make_pqfile_cols(label = label)
        df    = df.join(op_df, how = "left", on = "id")
        df    = df.select(pl.all().shrink_dtype())
        del op_df
        
        PrintColor(f"---> Shape = {df.shape} - memory usage {df.estimated_size('mb') :.3f} MB", 
                   color = Fore.CYAN
                  )
        return df, cat_cols
        

In [ ]:
# Load data
train = pl.read_csv(DATA_DIR / "train.csv").drop(['PCIAT-Season',], strict=False)#.drop(['Basic_Demos-Enroll_Season', 'CGAS-Season'], strict=False)
test = pl.read_csv(DATA_DIR / "test.csv").drop(['PCIAT-Season'], strict=False)#.drop(['Basic_Demos-Enroll_Season', 'CGAS-Season'], strict=False)

# Ensure 'id' is a string in both DataFrames
train = train.with_columns(pl.col('id').cast(pl.Utf8))
test = test.with_columns(pl.col('id').cast(pl.Utf8))

pp = Preprocessor()

# Use StringCache to handle categoricals consistently
with pl.StringCache():
    train, cat_cols = pp.pp_data(train, "Train")
    test, _ = pp.pp_data(test, "Test", cat_cols)

# Ensure data types match between train and test
for col in train.columns:
    if col in test.columns:
        train_dtype = train.schema[col]
        test_dtype = test.schema[col]
        if train_dtype != test_dtype:
            # Cast test column to train's data type
            test = test.with_columns(pl.col(col).cast(train_dtype))
    else:
        # Column in train but not in test, fill with nulls in test
        test = test.with_columns(pl.lit(None).cast(train.schema[col]).alias(col))

# Reorder test columns to match train
test = test.select(train.columns)

# Concatenate train and test
train_test = pl.concat([train, test], how="vertical")

IS_TEST = test.height <= 100

assert_frame_equal(train, train_test[: train.height].select(train.columns))
assert_frame_equal(test, train_test[train.height :].select(test.columns))

IS_TEST = test.height <= 100

assert_frame_equal(train, train_test[: train.height].select(train.columns))
assert_frame_equal(test, train_test[train.height :].select(test.columns))

In [ ]:
# Concatenate train and test
train_test = pl.concat([train, test], how="vertical")

# Cast string columns to categorical
train_test = train_test.with_columns(cs.string().cast(pl.Categorical).fill_null("NAN"))

# Get cat_features and num_features
cat_features = train_test.select(FEATURE_COLS).select(cs.categorical()).columns
print("Cat Features selected:")
print(cat_features)  # Should be none

num_features = train_test.select(FEATURE_COLS).select(cs.numeric()).columns
print("Numerical Features selected:")
print(num_features)

# Impute missing values in numerical features
imputation_strategy='mean'  # imputation_strategy (str): Strategy for imputing missing values ('mean', 'median', 'most_frequent', etc.).
imputer = SimpleImputer(strategy=imputation_strategy)
train_test[num_features] = imputer.fit_transform(train_test[num_features])

train = train_test[: train.height]
test = train_test[train.height :]

# ignore rows with null values in TARGET_COLS
train_without_null = train_test.drop_nulls(subset=TARGET_COLS)
X = train_without_null.select(FEATURE_COLS)
X_test = test.select(FEATURE_COLS)
y = train_without_null.select(TARGET_COLS)
y_sii = y.get_column("sii").to_numpy()  # ground truth



# Tubotubo's Quadratic Weighted Kappa metric & Optuna optimizer

In [ ]:
class MultiTargetQWK(MultiTargetCustomMetric):
    def get_final_error(self, error, weight):
        return np.sum(error)  # / np.sum(weight)

    def is_max_optimal(self):
        # if True, the bigger the better
        return True

    def evaluate(self, approxes, targets, weight):
        # approxes: 予測値 (shape: [ターゲット数, サンプル数])
        # targets: 実際の値 (shape: [ターゲット数, サンプル数])
        # weight: サンプルごとの重み (Noneも可)

        approx = np.clip(approxes[-1], 0, 3).round().astype(int)
        target = targets[-1]

        qwk = cohen_kappa_score(target, approx, weights="quadratic")

        return qwk, 1

    def get_custom_metric_name(self):
        return "MultiTargetQWK"


class OptimizedRounder:
    """
    A class for optimizing the rounding of continuous predictions into discrete class labels using Optuna.
    The optimization process maximizes the Quadratic Weighted Kappa score by learning thresholds that separate
    continuous predictions into class intervals.

    Args:
        n_classes (int): The number of discrete class labels.
        n_trials (int, optional): The number of trials for the Optuna optimization. Defaults to 100.

    Attributes:
        n_classes (int): The number of discrete class labels.
        labels (NDArray[np.int_]): An array of class labels from 0 to `n_classes - 1`.
        n_trials (int): The number of optimization trials.
        metric (Callable): The Quadratic Weighted Kappa score metric used for optimization.
        thresholds (List[float]): The optimized thresholds learned after calling `fit()`.

    Methods:
        fit(y_pred: NDArray[np.float_], y_true: NDArray[np.int_]) -> None:
            Fits the rounding thresholds based on continuous predictions and ground truth labels.

            Args:
                y_pred (NDArray[np.float_]): Continuous predictions that need to be rounded.
                y_true (NDArray[np.int_]): Ground truth class labels.

            Returns:
                None

        predict(y_pred: NDArray[np.float_]) -> NDArray[np.int_]:
            Predicts discrete class labels by rounding continuous predictions using the fitted thresholds.
            `fit()` must be called before `predict()`.

            Args:
                y_pred (NDArray[np.float_]): Continuous predictions to be rounded.

            Returns:
                NDArray[np.int_]: Predicted class labels.

        _normalize(y: NDArray[np.float_]) -> NDArray[np.float_]:
            Normalizes the continuous values to the range [0, `n_classes - 1`].

            Args:
                y (NDArray[np.float_]): Continuous values to be normalized.

            Returns:
                NDArray[np.float_]: Normalized values.

    References:
        - This implementation uses Optuna for threshold optimization.
        - Quadratic Weighted Kappa is used as the evaluation metric.
    """

    def __init__(self, n_classes: int, n_trials: int = 100):
        self.n_classes = n_classes
        self.labels = np.arange(n_classes)
        self.n_trials = n_trials
        self.metric = partial(cohen_kappa_score, weights="quadratic")

    def fit(self, y_pred: NDArray[np.float_], y_true: NDArray[np.int_]) -> None:
        y_pred = self._normalize(y_pred)

        def objective(trial: optuna.Trial) -> float:
            thresholds = []
            for i in range(self.n_classes - 1):
                low = max(thresholds) if i > 0 else min(self.labels)
                high = max(self.labels)
                th = trial.suggest_float(f"threshold_{i}", low, high)
                thresholds.append(th)
            try:
                y_pred_rounded = np.digitize(y_pred, thresholds)
            except ValueError:
                return -100
            return self.metric(y_true, y_pred_rounded)

        optuna.logging.disable_default_handler()
        study = optuna.create_study(direction="maximize")
        study.optimize(
            objective,
            n_trials=self.n_trials,
        )
        self.thresholds = [study.best_params[f"threshold_{i}"] for i in range(self.n_classes - 1)]

    def predict(self, y_pred: NDArray[np.float_]) -> NDArray[np.int_]:
        assert hasattr(self, "thresholds"), "fit() must be called before predict()"
        y_pred = self._normalize(y_pred)
        return np.digitize(y_pred, self.thresholds)

    def _normalize(self, y: NDArray[np.float_]) -> NDArray[np.float_]:
        # normalize y_pred to [0, n_classes - 1]
        return (y - y.min()) / (y.max() - y.min()) * (self.n_classes - 1)

In [ ]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')


def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

# Start making models!

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from catboost import CatBoostRegressor
import numpy as np
import pandas as pd

# Updated SILLY MODEL with Polynomial Features and Imputation
class SillyManPolynomial:
    def __init__(self, cat_params, degree=2, include_bias=False):
        """
        Initializes the SillyManPolynomial model.

        Parameters:
        - degree (int): Degree of the polynomial features.
        - include_bias (bool): Whether to include a bias column.
        """
        self.poly = PolynomialFeatures(degree=degree, include_bias=include_bias)
        self.cat = CatBoostRegressor(**cat_params)
        
    def fit(self, x, y, eval_set, cat_features, verbose=False):
        """
        Fits the model with polynomial feature generation and imputation.

        Parameters:
        - x (pd.DataFrame): Training features.
        - y (pd.Series or pd.DataFrame): Training target.
        - eval_set (tuple): Validation set as (X_val, y_val).
        - cat_features (list): List of categorical feature names to exclude or handle separately.
        - verbose (bool): Verbosity flag.
        """
        self.cat_features = cat_features
        
        # Separate numerical features (assuming categorical features are to be excluded or handled separately)
        if cat_features:
            numerical = x.drop(columns=self.cat_features)
            val_numerical = eval_set[0].drop(columns=self.cat_features)
            categorical = x[self.cat_features]
            val_categorical = eval_set[0][self.cat_features]
        else:
            numerical = x.copy()
            val_numerical = eval_set[0]
            categorical = None

        # Apply Polynomial Feature Generation to numerical features
        X_train_poly = np.concatenate((numerical, self.poly.fit_transform(numerical)), axis=1)
        X_val_poly = np.concatenate((val_numerical, self.poly.transform(val_numerical)), axis=1)

        # If there are categorical features, append them to the polynomial features
        if categorical is not None:
            X_train_cat = categorical.values
            X_val_cat = val_categorical.values
            X_train_processed = np.hstack([X_train_poly, X_train_cat])
            X_val_processed = np.hstack([X_val_poly, X_val_cat])
            # Update cat_features indices to point to the categorical features appended at the end
            new_cat_features = list(range(X_train_poly.shape[1], X_train_processed.shape[1]))
        else:
            X_train_processed = X_train_poly
            X_val_processed = X_val_poly
            new_cat_features = []

        # Fit CatBoost
        self.cat.fit(
            X_train_processed,
            y,
            eval_set=(X_val_processed, eval_set[1]),
            cat_features=new_cat_features,  # Specify categorical features if any
            verbose=verbose
        )
        
    def predict(self, x):
        """
        Predicts using the fitted model.

        Parameters:
        - x (pd.DataFrame): Features for prediction.
        - cat_features (list): List of categorical feature names.

        Returns:
        - np.ndarray: Predictions.
        """
        # Separate numerical features
        if cat_features:
            numerical = x.drop(columns=self.cat_features)
            categorical = x[self.cat_features]
        else:
            numerical = x.copy()
            categorical = None
        
        # Apply Polynomial Feature Generation
        X_poly = np.concatenate((numerical, self.poly.transform(numerical)), axis=1)
        
        # If there are categorical features, append them
        if categorical is not None:
            X_cat = categorical.values
            X_processed = np.hstack([X_poly, X_cat])
        else:
            X_processed = X_poly

        return self.cat.predict(X_processed)


In [ ]:
# Cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=52)
models: list = []
y_pred = np.full((X.height, len(TARGET_COLS)), fill_value=np.nan)

train_S = []
test_S = []

for train_idx, val_idx in skf.split(X, y_sii):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Define your parameters
    cat_params = dict(
        loss_function="MultiRMSE",
        eval_metric="MultiRMSE",  # Ensure eval_metric is appropriate
        iterations=8000,  # Adjust as needed
        learning_rate=0.05,
        depth=5,
        early_stopping_rounds=50,
    )
    
    # Initialize and train model
    model = SillyManPolynomial(cat_params)
    #model = CatBoostRegressor(**params)
    model.fit(
        X_train.to_pandas(),
        y_train.to_pandas(),
        eval_set=(X_val.to_pandas(), y_val.to_pandas()),
        cat_features=cat_features,
        verbose=False,
    )
    models.append(model)
    
    # Predict
    y_train_pred = model.predict(X_train.to_pandas())
    y_val_pred = model.predict(X_val.to_pandas())
    y_pred[val_idx] = y_val_pred
    
    train_kappa = quadratic_weighted_kappa(y_train['sii'], y_train_pred[:, TARGET_COLS.index("sii")].round(0).astype(int))
    val_kappa = quadratic_weighted_kappa(y_val['sii'], y_val_pred[:, TARGET_COLS.index("sii")].round(0).astype(int))

    train_S.append(train_kappa)
    test_S.append(val_kappa)

assert np.isnan(y_pred).sum() == 0

# Optimize thresholds
optimizer = OptimizedRounder(n_classes=4, n_trials=300)
y_pred_total = y_pred[:, TARGET_COLS.index("PCIAT-PCIAT_Total")]
optimizer.fit(y_pred_total, y_sii)
y_pred_rounded = optimizer.predict(y_pred_total)

# Calculate QWK
qwk = cohen_kappa_score(y_sii, y_pred_rounded, weights="quadratic")
print(f"Cross-Validated QWK Score: {qwk}")

print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

KappaOPtimizer = minimize(evaluate_predictions,
                          x0=[0.5, 1.5, 2.5], args=(y['sii'], y_pred[:, TARGET_COLS.index("sii")]), 
                          method='Nelder-Mead') # Nelder-Mead | # Powell
assert KappaOPtimizer.success, "Optimization did not converge."

oof_tuned = threshold_Rounder(y_pred[:, TARGET_COLS.index("sii")], KappaOPtimizer.x)
tKappa = quadratic_weighted_kappa(y['sii'], oof_tuned)

PrintColor(f"----> || Optimized QWK SCORE :: {tKappa:.3f}", Fore.CYAN, Style.RESET_ALL)

In [ ]:
class AvgModel:
    def __init__(self, models: list[BaseEstimator]):
        self.models = models

    def predict(self, X: ArrayLike) -> NDArray[np.int_]:
        preds: list[NDArray[np.int_]] = []
        for model in self.models:
            pred = model.predict(X)
            preds.append(pred)

        return np.mean(preds, axis=0)

In [ ]:
avg_model = AvgModel(models)
test_pred = avg_model.predict(X_test.to_pandas())[:, TARGET_COLS.index("PCIAT-PCIAT_Total")]
test_pred_rounded = optimizer.predict(test_pred)
test.select("id").with_columns(
    pl.Series("sii", pl.Series("sii", test_pred_rounded)),
).write_csv("submission.csv")

In [ ]:
PrintColor("Success!", Fore.GREEN, Style.RESET_ALL)

In [ ]:
# HP TUNE OR BUSSSST
tr = 0

def objective(trial: optuna.Trial) -> float:
    global tr
    tr += 1
    print("TRIAL", tr)
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=52)
    models: list = []
    y_pred = np.full((X.height, len(TARGET_COLS)), fill_value=np.nan)
    
    train_S = []
    test_S = []
    
    for train_idx, val_idx in skf.split(X, y_sii):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Define your parameters
        cat_params = dict(
            loss_function="MultiRMSE",
            eval_metric="MultiRMSE",
            iterations=trial.suggest_int('iterations', 500, 12000),
            learning_rate=trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
            depth = trial.suggest_int('depth', 4, 10, step=1),
            early_stopping_rounds = trial.suggest_int('early_stopping_rounds', 10, 1000),
            bagging_temperature = trial.suggest_int('bagging_temperature', 0, 24, step=3),
            l2_leaf_reg=trial.suggest_loguniform('l2_leaf_reg', 1e-3, 20.0),
            random_strength=trial.suggest_int('random_strength', 1, 9, step=2),
            colsample_bylevel=trial.suggest_loguniform('colsample_bylevel', 0.1, 1.0),
            min_data_in_leaf=trial.suggest_loguniform('min_data_in_leaf', 1, 50),
        )
        
        # Initialize and train model
        model = SillyManPolynomial(cat_params)
        #model = CatBoostRegressor(**params)
        model.fit(
            X_train.to_pandas(),
            y_train.to_pandas(),
            eval_set=(X_val.to_pandas(), y_val.to_pandas()),
            cat_features=cat_features,
            verbose=False,
        )
        models.append(model)
        
        # Predict
        y_train_pred = model.predict(X_train.to_pandas())
        y_val_pred = model.predict(X_val.to_pandas())
        y_pred[val_idx] = y_val_pred
        
        train_kappa = quadratic_weighted_kappa(y_train['sii'], y_train_pred[:, TARGET_COLS.index("sii")].round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val['sii'], y_val_pred[:, TARGET_COLS.index("sii")].round(0).astype(int))
    
        train_S.append(train_kappa)
        test_S.append(val_kappa)
    
    assert np.isnan(y_pred).sum() == 0
    
    # Optimize thresholds
    optimizer = OptimizedRounder(n_classes=4, n_trials=300)
    y_pred_total = y_pred[:, TARGET_COLS.index("PCIAT-PCIAT_Total")]
    optimizer.fit(y_pred_total, y_sii)
    y_pred_rounded = optimizer.predict(y_pred_total)
    
    # Calculate QWK
    qwk = cohen_kappa_score(y_sii, y_pred_rounded, weights="quadratic")
    print(f"Cross-Validated QWK Score: {qwk}")
    
    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    mv = np.mean(test_S)
    print(f"Mean Validation QWK ---> {mv:.4f}")
    
    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y['sii'], y_pred[:, TARGET_COLS.index("sii")]), 
                              method='Nelder-Mead') # Nelder-Mead | # Powell
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(y_pred[:, TARGET_COLS.index("sii")], KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y['sii'], oof_tuned)
    
    PrintColor(f"----> || Optimized QWK SCORE :: {tKappa:.3f}", Fore.CYAN, Style.RESET_ALL)
    return mv

optuna.logging.enable_default_handler()
study = optuna.create_study(direction="maximize")
study.optimize(
    objective,
    n_trials=50,
    n_jobs=1
)

In [ ]:
print('Best Trial:')
trial = study.best_trial
print(f'  Value: {trial.value}')
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')